In [1]:
import os
from alive_progress import alive_bar
from openmm import app, unit, Vec3, ReaxffForce, Platform, LangevinIntegrator
from openmm.app import StateDataReporter, PDBFile, Modeller, ForceField, Simulation, PDBReporter
from utils.ReaxFFHelpers  import setup_reaxff_atoms, remove_extra_forces, get_nonbonded_forces, switch_nonbonded_force
from utils.TopologyTools import TopologyTools

In [2]:
# load protein and label chains as PROTEIN-0, PROTEIN-1, ...
protein = PDBFile("chig.pdb")
TopologyTools.add_chain_name("PROTEIN", protein.topology)

Warning, renaming chain, old chain id was  A .


In [3]:
# The modeller instance holds the topology and the positions openmm is going to use in the simulation
# and has useful functions like addSolvent and addHydrogens
modeller = Modeller(protein.topology, protein.positions)

In [4]:
# protein and water forcefield
forcefield = ForceField("amber14/protein.ff14SB.xml", "amber14/tip3p.xml")

In [5]:
#solvate the system in an 8x8x8 nm^3 water box with tip3p water
modeller.addSolvent(forcefield, model="tip3p", boxSize=4*Vec3(1, 1, 1 )*unit.nanometers)

In [6]:
# create the system
system = forcefield.createSystem(modeller.topology, #
                                 nonbondedCutoff=1.1*unit.nanometers,
                                switchDistance=1.0*unit.nanometers,
                                 hydrogenMass=4*unit.amu,
                                 rigidWater=True,
                                 constraints=app.AllBonds,
                                 nonbondedMethod=app.CutoffPeriodic,
                                 removeCMMotion=False
                                )

In [7]:
reax_atoms = []
link_atom_1 = None
link_atom_2 = None
backbone = ["N", "CA", "C", "O", "HA", "H"]

for chain in modeller.topology.chains():
    if chain.id.startswith("PROTEIN"):
        for residue in chain.residues():
            if residue.name == "MET":
                for atom in residue.atoms():
                    if atom.name not in backbone:
                        reax_atoms.append(atom.index)
                    if atom.name == "CA":
                        link_atom_1 = atom.index
                    if atom.name == "CB":
                        link_atom_2 = atom.index

atom_symbols = [atom.element.symbol for atom in modeller.topology.atoms()]

In [8]:
# create a list of the reaxff atom indices and all of the atom symbols
atom_symbols = [atom.element.symbol for atom in modeller.topology.atoms()]

In [9]:
# define the reaxff force
# ffield.reaxff: force field parameters
# control: defines control parameters, like charge equilibriation algorithm, cutoffs, etc.
force = ReaxffForce("/home/babaid/repos/reaxff-mm-examples/ffield.reaxff", 
                            "/home/babaid/repos/reaxff-mm-examples/control", 1)


system.addForce(force)
force.setForceGroup(1)

In [10]:
remove_extra_forces(system, reax_atoms, True)

Number of bond forces before update:  0
Number of bond forces after update:  0
Number of angle forces before update:  255
Number of angle forces after update:  233
Number of torsion forces before update:  452
Number of torsion forces after update:  410


In [11]:
reaxff_nonbonded_force = get_nonbonded_forces(system, reax_atoms, atom_symbols, force, [link_atom_1])

In [12]:
force.addLink(link_atom_2, link_atom_1)

In [13]:
for force in system.getForces():
    print(force.getName(), ": ", force.getForceGroup() )

HarmonicBondForce :  0
PeriodicTorsionForce :  0
NonbondedForce :  0
HarmonicAngleForce :  0
ReaxFFForce :  1
HarmonicAngleForce :  1
HarmonicBondForce :  1
PeriodicTorsionForce :  1


In [14]:
ncs = system.getNumConstraints()
for i in range(system.getNumConstraints() - 1, -1, -1):
    particle1, particle2, dist = system.getConstraintParameters(i)
    if (particle1 in reax_atoms) and (particle2 in reax_atoms):
        system.removeConstraint(i) 
print("Removed ", ncs-system.getNumConstraints(), " constraints.")

Removed  10  constraints.


In [15]:
switch_nonbonded_force(system, reaxff_nonbonded_force)

In [16]:
#the list of indices of not reactive atoms
non_reax_atoms = [atom.index for atom in modeller.topology.atoms() if atom.index not in reax_atoms]
print("Number of MM atoms: ", len(non_reax_atoms))

Number of MM atoms:  6096


# Restraints and SMD setup

In [17]:
# use CUDA for computation (other options: OpenCL, CPU, Reference) 
platform = Platform.getPlatform("CUDA")
platform.getName()

'CUDA'

In [18]:
# Langevin dynamics, at 300K with a friction coefficient of 10/ps and timestep of 0.25fs
integrator = LangevinIntegrator(300*unit.kelvin, 10/unit.picosecond, 1*unit.femtosecond)

simulation = Simulation(modeller.topology, system, integrator, platform)

simulation.context.setPositions(modeller.positions)
simulation.reporters.append(PDBReporter('NVT.pdb', 1000))

In [19]:
# this function modifies the bond dissociation energy of all the hydrogen bonds
#modify_reaxff_dissociation_energy_with_ratio("../ffield.reaxff", "ffield.reaxff", 1.0)
integrator.setIntegrationForceGroups({1})

In [ ]:
simulation.minimizeEnergy()

In [ ]:
# STEERED MD STAGE 1: NVT HEATUP
# Parameters. Time -> 1ns T -> 300K
temperature = 0*unit.kelvin
dt = 3*unit.kelvin
print("NVT Heatup")
with alive_bar(100, force_tty=True) as bar:
    for i in range(100):
        temperature += dt
        integrator.setTemperature(temperature)
        simulation.step(params["steps"]/100)
        bar()
print("Done.")

In [ ]:
!jupyter nbconvert --to script chignolin_mutation-bbonly.ipynb